Step 4

In [1]:
import pandas as pd
import os

In [2]:
import neo4j

In [3]:
from neo4j import GraphDatabase

In [4]:
from graphdatascience import GraphDataScience

In [5]:
NEO4J_URI = os.environ.get("NEO4J_URI", "bolt://localhost:7687")

In [6]:
NEO4J_AUTH = ( "neo4j",  "june15kent")

In [7]:
driver = GraphDatabase.driver(NEO4J_URI, auth=NEO4J_AUTH)

In [8]:
gds = GraphDataScience(NEO4J_URI, auth=NEO4J_AUTH)

Step 5

In [9]:
test_query = '''
MATCH (n) 
RETURN n
LIMIT 10
'''

In [10]:
pii_query = '''
MATCH (c:Client)-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-> (n) <-[ :HAS_EMAIL |:HAS_PHONE |:HAS_SSN ]-(d:Client)
WHERE id(c) < id(d)
RETURN c.id, d.id, count(*) AS freq
ORDER BY freq DESC
'''

In [11]:
gds.run_cypher(pii_query)

C:\Users\kentp\anaconda3\envs\cs350\Lib\site-packages\graphdatascience\query_runner\neo4j_query_runner.py:234: RuntimeWarning: The semantics of using colon in the separation of alternative relationship types will change in a future version. (Please use ':HAS_EMAIL|HAS_PHONE|HAS_SSN' instead)
  warnings.warn(warning)


,c.id,d.id,freq
0,4952527271473904,4816336012071985,3
1,4883445100935916,4708373581412325,3
2,4658150168863397,4100374538108184,3
3,4673951123644611,4795773320377768,3
4,4192214340630620,4912097363222923,3
...,...,...,...
754,4910140986334626,4114683318919154,1
755,4454780847105236,4210575070378533,1
756,4721862020593706,4210575070378533,1
757,4445521165797820,4210575070378533,1


Step 6

In [1]:
CALL gds.graph.project(
    'clientClusters' ,
    { Client: {
        label: 'Client' }
    },
    { SHARED_PII: {
        type: 'SHARED_PII',
        orientation: 'UNDIRECTED',
        properties: {
        count: {
        property: 'count' }
            }
        }
    }
)
YIELD graphName, nodeCount, relationshipCount

SyntaxError: invalid syntax (22362033.py, line 1)

Step 9

In [ ]:
MATCH (c:Client) WHERE c.secondPartyFraudRing is NOT NULL
WITH collect(c) as clients

MATCH (n) WHERE n:Email OR n:Phone OR n:SSN

WITH clients, collect(n) AS piis
WITH clients + piis AS nodes

MATCH (c:Client) -[:HAS_EMAIL | :HAS_PHONE | :HAS_SSN]->(p)
WHERE c.secondPartyFraudRing is NOT NULL

WITH nodes, collect({source: c, target: p}) as relationships

In [ ]:
CALL gds.graph.project.cypher(
    'similarity',
    "UNWIND $nodes as n
        RETURN id(n) AS id,labels(n) AS labels",
    "UNWIND $relationships as r
        RETURN id(r['source']) AS source, id(r['target']) AS target,
    'HAS_PII' as type",
    { parameters:
        { nodes: nodes,
            relationships: relationships }
    }
)
YIELD graphName, nodeCount, relationshipCount
RETURN graphName, nodeCount, relationshipCount

Step 10

In [ ]:
CALL gds.nodeSimilarity.mutate(
  'similarity',
  {
    mutateProperty: 'jaccardScore',
    mutateRelationshipType: 'SIMILAR_TO',
    topK: 15
  }
)
YIELD
  preProcessingMillis,
  computeMillis,
  mutateMillis,
  postProcessingMillis,
  relationshipsWritten,
  nodesCompared,
  similarityDistribution,
  configuration
